In [43]:
# imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import gc

In [44]:
matrix_array = np.load('Data/matrix_array_20_normalized.npy')
answer_array = np.load('Data/answer_array_20.npy')

In [45]:
matrix_array.shape, answer_array.shape    

((580843, 20, 19), (580843, 3))

In [46]:
from sklearn.model_selection import train_test_split

X = matrix_array
y = answer_array


X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size=0.8, random_state=1, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

del matrix_array, answer_array, X, y, X_train ,X_temp, y_train, y_temp, X_valid, y_valid
gc.collect()

2989

In [47]:
from datetime import datetime, timezone

# Convert ISO 8601 date strings to Unix timestamp (milliseconds)
def iso_to_unix(iso_str):
    dt = datetime.strptime(iso_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)  # Convert to milliseconds

def unix_to_iso(unix_timestamp_ms):
    # Convert milliseconds to seconds
    unix_timestamp_s = unix_timestamp_ms / 1000
    # Create a datetime object from the Unix timestamp
    dt = datetime.utcfromtimestamp(unix_timestamp_s)
    # Format the datetime object as an ISO 8601 date string
    iso_str = dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    return iso_str

In [48]:
# data = pd.read_csv('Data/historical_BTCUSDT_3min_data.csv')
# data_2023 = data[data['Open time']> iso_to_unix('2023-01-01T00:00:00Z')]
# data_2023.to_csv('Data/2023_BTC_data.csv', index=False)
# data_2023 = data_2023.drop(['Close time', 'Ignore'], axis=1)
# data_2023_no_opentime = data_2023.drop(['Open time'], axis=1)
# data_2023

data_2023 = pd.read_csv('Data/until_2023_BTCUSDT_3min_data.csv')
data_2023 = data_2023.drop(['Ignore', 'Close time'], axis=1)
data_2023_no_opentime = data_2023.drop(['Open time'], axis=1)

data_2024 = pd.read_csv('Data/2024_BTCUSDT_3min_data.csv')
data_2024 = data_2024.drop(['Ignore', 'Close time'], axis=1)
data_2024_no_opentime = data_2024.drop(['Open time'], axis=1)

In [49]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2023['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023
    data_2023[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2023['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2023['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023
data_2023[f'MA{BB_window_size}'] = BB_moving_avg
data_2023[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2023[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2023 = data_2023.dropna().reset_index(drop=True)
data_2023

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,MA5,MA10,MA20,MA30,MA60,MA120,MA240,MA90,Upper_Band90,Lower_Band90
0,1598961420000,11844.99,11869.81,11844.98,11868.99,283.693945,3.364212e+06,5221,157.389799,1.866334e+06,11876.222,11887.680,11890.2970,11901.974000,11908.867000,11891.066667,11772.887833,11907.037000,11928.547284,11885.526716
1,1598961600000,11868.98,11869.39,11840.78,11842.28,302.129538,3.581858e+06,4557,112.710021,1.336253e+06,11864.576,11881.721,11887.1830,11899.154333,11907.887167,11891.445333,11773.617667,11906.627778,11929.008071,11884.247485
2,1598961780000,11842.29,11851.93,11842.29,11845.86,186.147149,2.205477e+06,3935,103.918062,1.231236e+06,11856.386,11876.508,11884.0755,11896.434667,11907.128000,11891.898333,11774.453792,11906.261111,11929.360231,11883.161991
3,1598961960000,11845.70,11860.00,11844.87,11858.40,130.267852,1.544293e+06,3080,65.908395,7.812655e+05,11852.104,11873.253,11882.5455,11893.833667,11906.581000,11892.449833,11775.367458,11905.984111,11929.506898,11882.461324
4,1598962140000,11858.39,11863.44,11852.42,11854.57,221.540694,2.626956e+06,3704,113.076940,1.340857e+06,11854.020,11868.561,11880.4120,11891.627000,11906.289833,11893.071333,11776.265625,11905.590444,11929.673537,11881.507352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583437,1704066480000,42267.89,42273.63,42247.87,42249.06,70.344940,2.972979e+06,1683,39.426970,1.666304e+06,42241.138,42210.400,42229.0150,42325.899333,42449.714167,42534.281167,42529.798000,42504.295889,42661.796127,42346.795651
583438,1704066660000,42249.06,42249.07,42221.22,42228.85,36.702670,1.549980e+06,1372,10.803520,4.562417e+05,42242.046,42219.978,42226.8870,42315.772000,42443.618333,42531.196667,42528.609917,42499.938889,42659.584200,42340.293578
583439,1704066840000,42228.86,42240.93,42222.10,42240.93,30.495130,1.287764e+06,1373,18.944770,7.999862e+05,42245.562,42227.066,42223.7170,42305.496333,42438.500167,42528.170750,42527.442583,42495.418556,42656.583736,42334.253375
583440,1704067020000,42240.92,42283.59,42240.92,42283.58,56.903090,2.405381e+06,1847,34.161340,1.444019e+06,42254.060,42237.244,42225.0020,42295.832333,42433.926333,42525.398667,42526.464333,42491.090889,42652.660964,42329.520814


In [50]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2024['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023
    data_2024[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2024['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2024['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023
data_2024[f'MA{BB_window_size}'] = BB_moving_avg
data_2024[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2024[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2024 = data_2024.dropna().reset_index(drop=True)
data_2024

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,MA5,MA10,MA20,MA30,MA60,MA120,MA240,MA90,Upper_Band90,Lower_Band90
0,1704110220000,42699.98,42699.99,42690.20,42690.20,17.11771,7.308644e+05,1164,11.49433,4.907639e+05,42661.220,42643.637,42677.0035,42685.296333,42659.149167,42547.639333,42492.441625,42604.693556,42693.105219,42516.281892
1,1704110400000,42690.21,42739.22,42690.20,42709.20,49.46675,2.113035e+06,3419,34.70596,1.482377e+06,42677.110,42652.034,42675.4085,42686.722667,42661.478333,42551.540167,42494.040375,42607.039111,42695.391295,42518.686927
2,1704110580000,42709.19,42709.20,42691.10,42693.44,18.60200,7.942602e+05,1682,6.19812,2.646414e+05,42686.954,42656.056,42672.7600,42686.891000,42663.819667,42554.826500,42495.224708,42609.241556,42697.244919,42521.238192
3,1704110760000,42693.44,42693.45,42622.00,42625.99,54.82226,2.338596e+06,2040,13.06340,5.572799e+05,42683.764,42655.655,42666.1615,42684.931000,42665.104000,42557.624917,42496.082708,42610.663667,42697.879819,42523.447514
4,1704110940000,42625.99,42653.39,42625.99,42642.92,13.25919,5.654427e+05,1402,7.72162,3.292666e+05,42672.350,42658.546,42661.0675,42683.028000,42666.686500,42560.630000,42496.764375,42612.674333,42698.498581,42526.850086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48717,1712879280000,70079.02,70084.00,70052.00,70084.00,25.26184,1.770023e+06,1312,12.05860,8.448591e+05,70060.954,70012.212,70074.4965,70113.641000,70190.209833,70252.641917,70194.043167,70284.125556,70476.002743,70092.248369
48718,1712879460000,70084.00,70105.80,70072.01,70105.80,18.87734,1.322997e+06,1219,9.53525,6.682760e+05,70075.114,70021.762,70069.7865,70109.332333,70184.461333,70251.550417,70193.750292,70282.067222,70474.860726,70089.273718
48719,1712879640000,70105.80,70135.55,70072.81,70072.82,21.76637,1.525950e+06,1427,10.33463,7.245631e+05,70079.680,70043.046,70060.4550,70103.941667,70177.470167,70249.724667,70193.954917,70279.898556,70473.945754,70085.851357
48720,1712879820000,70072.81,70072.82,70006.21,70006.23,13.44942,9.418671e+05,983,1.34757,9.437439e+04,70069.574,70047.470,70053.2035,70095.816000,70169.247000,70248.076667,70194.051042,70276.481111,70472.621629,70080.340594


In [51]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2023_no_opentime['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023_no_opentime
    data_2023_no_opentime[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2023_no_opentime['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2023_no_opentime['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023_no_opentime
data_2023_no_opentime[f'MA{BB_window_size}'] = BB_moving_avg
data_2023_no_opentime[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2023_no_opentime[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2023_no_opentime = data_2023_no_opentime.dropna().reset_index(drop=True)
data_2023_no_opentime.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60',
       'MA120', 'MA240', 'MA90', 'Upper_Band90', 'Lower_Band90'],
      dtype='object')

In [52]:
# Define the window sizes for the moving averages
windows = [5, 10, 20, 30, 60, 120, 240]
# Define the window size and standard deviation multiplier for the Bollinger Bands
BB_window_size = 90
BB_std_multiplier = 1


# ADDING MA
for window in windows:
    # Calculate the moving average
    moving_avg = data_2024_no_opentime['Close'].rolling(window).mean()
    # Add the moving average as a new column to the data_2023_no_opentime
    data_2024_no_opentime[f'MA{window}'] = moving_avg

# Calculate the moving average and standard deviation
BB_moving_avg = data_2024_no_opentime['Close'].rolling(BB_window_size).mean()
BB_std_dev = data_2024_no_opentime['Close'].rolling(BB_window_size).std()

# ADDING BB
# Calculate the Bollinger Bands
BB_upper_band = BB_moving_avg + (BB_std_multiplier * BB_std_dev)
BB_lower_band = BB_moving_avg - (BB_std_multiplier * BB_std_dev)

# Add the Bollinger Bands and moving average as new columns to the data_2023_no_opentime
data_2024_no_opentime[f'MA{BB_window_size}'] = BB_moving_avg
data_2024_no_opentime[f'Upper_Band{BB_window_size}'] = BB_upper_band
data_2024_no_opentime[f'Lower_Band{BB_window_size}'] = BB_lower_band





# drop NaN values
data_2024_no_opentime = data_2024_no_opentime.dropna().reset_index(drop=True)
data_2024_no_opentime.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60',
       'MA120', 'MA240', 'MA90', 'Upper_Band90', 'Lower_Band90'],
      dtype='object')

In [53]:
# Initialize empty lists to hold the data
matrix_list = []


for i in range(len(data_2023) - 19):  # Ensure there are 20 rows available
    # Create a matrix of 20 rows
    matrix = data_2023.iloc[i:i+20].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2023_matrix_20 = np.array(matrix_list)


In [54]:
matrix_list = []


for i in range(len(data_2024) - 19):  # Ensure there are 20 rows available
    # Create a matrix of 20 rows
    matrix = data_2024.iloc[i:i+20].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2024_matrix_20 = np.array(matrix_list)

In [55]:
# Initialize empty lists to hold the data
matrix_list = []


for i in range(len(data_2023_no_opentime) - 19):  # Ensure there are 20 rows available
    # Create a matrix of 20 rows
    matrix = data_2023_no_opentime.iloc[i:i+20].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2023_no_opentime_matrix_20 = np.array(matrix_list)


In [56]:
# Initialize empty lists to hold the data
matrix_list = []


for i in range(len(data_2024_no_opentime) - 19):  # Ensure there are 20 rows available
    # Create a matrix of 20 row
    matrix = data_2024_no_opentime.iloc[i:i+20].values
    matrix_list.append(matrix)

# Convert the lists to numpy arrays
data_2024_no_opentime_matrix_20 = np.array(matrix_list)


In [57]:
data_2023_matrix_20.shape, data_2023_no_opentime_matrix_20.shape, data_2024_matrix_20.shape, data_2024_no_opentime_matrix_20.shape

((583423, 20, 20), (583423, 20, 19), (48703, 20, 20), (48703, 20, 19))

In [58]:
np.save('Data/data_2023_matrix_20.npy', data_2023_matrix_20)
np.save('Data/data_2024_matrix_20.npy', data_2024_matrix_20)

## RFC model

In [59]:
import joblib

# Assuming your model is named RFC
RFC = joblib.load('Models/RFC_model_rank_1.pkl')
scaler = joblib.load('Scalers/StandardScaler_20.pkl')

In [60]:
# scale the data

# Assuming data is your 600k matrices concatenated into a single 3D numpy array of shape (600000, 20, 19)
temp = data_2023_no_opentime_matrix_20.reshape(-1, 19)  # Reshape to 2D for standardization
temp_normalized = scaler.transform(temp)

# Reshape back to 3D
data_2023_matrix_20_normalized = temp_normalized.reshape(-1, 20, 19)

data_2023_matrix_20_normalized_flattened = np.array([x.flatten() for x in data_2023_matrix_20_normalized])

y_pred = RFC.predict(data_2023_matrix_20_normalized_flattened)

np.save('Data/RFC_20_2023_prediction.npy', y_pred)

[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 144 tasks      | elapsed:    1.6s
[Parallel(n_jobs=28)]: Done 300 out of 300 | elapsed:    3.2s finished


In [61]:
# scale the data

# Assuming data is your 600k matrices concatenated into a single 3D numpy array of shape (600000, 20, 19)
temp = data_2024_no_opentime_matrix_20.reshape(-1, 19)  # Reshape to 2D for standardization
temp_normalized = scaler.transform(temp)

# Reshape back to 3D
data_2024_matrix_20_normalized = temp_normalized.reshape(-1, 20, 19)

data_2024_matrix_20_normalized_flattened = np.array([x.flatten() for x in data_2024_matrix_20_normalized])

y_pred = RFC.predict(data_2024_matrix_20_normalized_flattened)

np.save('Data/RFC_20_2024_prediction.npy', y_pred)

[Parallel(n_jobs=28)]: Using backend ThreadingBackend with 28 concurrent workers.
[Parallel(n_jobs=28)]: Done 144 tasks      | elapsed:    0.1s
[Parallel(n_jobs=28)]: Done 300 out of 300 | elapsed:    0.2s finished


## LSTM model

In [69]:
# imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchsummary import summary
from tqdm import  tqdm

import os, json, gc, io, joblib
from contextlib import redirect_stdout

from sklearn.model_selection import train_test_split

In [70]:
# Custom LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()

        # Multi-layer LSTM
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=3, batch_first=True, dropout=0.2)

        # Flatten layer 

        # Define Linear layers with Batch Normalization, GELU, and Dropout
        self.linear1 = nn.Linear(hidden_dim, 64)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.gelu1 = nn.GELU()
        self.dropout1 = nn.Dropout(0.2)

        self.linear2 = nn.Linear(64, 32)
        self.batch_norm2 = nn.BatchNorm1d(32)
        self.gelu2 = nn.GELU()
        self.dropout2 = nn.Dropout(0.2)

        self.linear3 = nn.Linear(32, 16)
        self.batch_norm3 = nn.BatchNorm1d(16)
        self.gelu3 = nn.GELU()

        # Output layer
        self.output_layer = nn.Linear(16, output_dim)

    def forward(self, x):
        # Pass input through LSTM layers
        # print("Output type1:", type(x))
        (lstm_out, _) = self.lstm(x)

        # Taking the output of the last time step
        # print("Output type2:", type(x))
        x = lstm_out[:, -1, :]

        # Pass through Linear layers
        x = self.linear1(x)
        x = self.batch_norm1(x)
        x = self.gelu1(x)
        x = self.dropout1(x)

        x = self.linear2(x)
        x = self.batch_norm2(x)
        x = self.gelu2(x)
        x = self.dropout2(x)

        x = self.linear3(x)
        x = self.batch_norm3(x)
        x = self.gelu3(x)

        # Output layer
        x = self.output_layer(x)
        
        return x 


In [75]:
# Initialize the model
model = LSTMModel(input_dim=19, hidden_dim=128, output_dim=3)

state_dict = torch.load('Models/LSTM/LSTM_model_state_dict_custominitialize.pth')
model.load_state_dict(state_dict)

# Setting device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
model = model.to(device)

In [76]:
import joblib

scaler = joblib.load('Scalers/StandardScaler_20.pkl')

# scale the data

# Assuming data is your 600k matrices concatenated into a single 3D numpy array of shape (600000, 20, 19)
temp = data_2024_no_opentime_matrix_20.reshape(-1, 19)  # Reshape to 2D for standardization
temp_normalized = scaler.transform(temp)

# Reshape back to 3D
data_2024_matrix_20_normalized = temp_normalized.reshape(-1, 20, 19)

In [77]:
data_2024_matrix_20_normalized.shape

(48703, 20, 19)

In [78]:
X = data_2024_matrix_20_normalized

X_tensor = torch.tensor(X, dtype=torch.float32)


In [79]:
import torch
import pandas as pd

# Ensure the model is in evaluation mode
model.eval()

# Move tensor to the same device as model
X_tensor = X_tensor.to(device)

# No need to track gradients for prediction
with torch.no_grad():
    # Predict
    predictions = model(X_tensor)

    # Apply softmax to convert logits to probabilities
    probabilities = torch.softmax(predictions, dim=1)  # Assuming output_dim corresponds to class counts
    # If you need class labels instead of probabilities:
    predicted_labels = torch.argmax(predictions, dim=1)

# Move predictions back to cpu for converting to numpy array
# probabilities = probabilities.cpu().numpy()
predicted_labels = predicted_labels.cpu().numpy()  # Uncomment if using argmax
print(predicted_labels)

np.save('Data/LSTM_2024_predictions.npy', predicted_labels)

[0 0 1 ... 2 2 2]
